In [48]:
import multiprocess as mp
import tqdm
import geopandas as gpd
import pandas as pd
import shapely
import fiona

### To do
- reduce coordinate precision.
- use vectorisation to loop over centroids

Load in the isochrones for the East Midlands

In [2]:
df = gpd.read_file('processed/east_midlands.geojson')

In [5]:
df

,iso_id,id,min,max,center,geometry
0,1,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.75054 55.74519, -2.07269 55..."
1,2,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.73652 55.71385, -2.05868 55..."
2,3,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.72142 55.71668, -2.04358 55..."
3,4,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.75198 55.70471, -2.07414 55..."
4,5,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.72179 55.70886, -2.04394 55..."
...,...,...,...,...,...,...
10632,10633,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.09036 54.74783, -1.41252 54..."
10633,10634,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.06124 54.72580, -1.38340 54..."
10634,10635,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.04668 54.73207, -1.36884 54..."
10635,10636,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.03212 54.74282, -1.35428 54..."


Load in the centroids from the population grid

In [3]:
centroids = gpd.read_file('data/gb_centroid.geojson')

In [7]:
centroids

,grid_id,population,geometry
0,1,24,POINT (-4.44691 53.42219)
1,2,15,POINT (-4.43188 53.42250)
2,3,10,POINT (-4.38678 53.42341)
3,4,32,POINT (-4.49149 53.41228)
4,5,121,POINT (-4.47646 53.41259)
...,...,...,...
117804,117805,7,POINT (-4.90856 54.66161)
117805,117806,0,POINT (-4.89308 54.66198)
117806,117807,2,POINT (-4.89244 54.65300)
117807,117808,8,POINT (-4.89180 54.64403)


Calculate the total population of the UK

In [4]:
totalpop=centroids['population'].sum()
totalpop

60553399

In [5]:
# define a function that takes a isochrone and loops through all the centroids to see if that centroid is within the isochrone. If it does, it adds the population to a running total.
def inthegoldentriangle(isochrone):
#     print(isochrone)
    population=0
    for i,centroid in centroids.iterrows():
        if(centroid['geometry'].within(isochrone)):
            population=population+centroid['population']
    if(population/totalpop>0.9):
        return True
    else:
        return False
    

In [6]:
# test it
inthegoldentriangle(df['geometry'][4])

False

In [7]:
# run the function across all the isochrones in parallel, save it to a list called output
pool=mp.Pool(8)
output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle, df['geometry']), total=len(df['geometry'])))
pool.close()
pool.join()

<ipython-input-7-355dd9e1457b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle, df['geometry']), total=len(df['geometry'])))


  0%|          | 0/10637 [00:00<?, ?it/s]

In [9]:
output

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [81]:
# find out how many squares are within the golden triangle
sum(x==True for x in output)

3

In [12]:
# Add the column to the isochrones
df['inside']=output

In [14]:
# Here's all the isochrones that are within the golden triangle
df[df['inside']==True]

,iso_id,id,min,max,center,geometry,inside
7985,7986,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.38911 55.42207, -1.71126 55...",True
8067,8068,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.40399 55.40869, -1.72615 55...",True
8068,8069,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.71135 55.33613, -1.88010 55...",True


In [15]:
# use Henry's lookup
lookup=gpd.read_file('data/east_midlands_lookup.geojson')

In [31]:
# find the centre points of the isochrones
goldentriangle=lookup[(lookup['iso_id']==7986) | (lookup['iso_id']==8068) | (lookup['iso_id']==8069)]
goldentriangle

,iso_id,id,min,max,center,population,lon,lat,geometry
7985,7986,1,0.0,240.0,120.0,138,-1.550185,52.691332,"MULTIPOLYGON (((-1.38911 55.42207, -1.71126 55..."
8067,8068,1,0.0,240.0,120.0,12,-1.565069,52.682398,"MULTIPOLYGON (((-1.40399 55.40869, -1.72615 55..."
8068,8069,1,0.0,240.0,120.0,21,-1.550278,52.682343,"MULTIPOLYGON (((-1.71135 55.33613, -1.88010 55..."


In [77]:
# make a geodataframe with the points
gdf = gpd.GeoDataFrame(goldentriangle, geometry=gpd.points_from_xy(goldentriangle.lon,goldentriangle.lat))
gdf

,iso_id,id,min,max,center,population,lon,lat,geometry
7985,7986,1,0.0,240.0,120.0,138,-1.550185,52.691332,POINT (-1.55019 52.69133)
8067,8068,1,0.0,240.0,120.0,12,-1.565069,52.682398,POINT (-1.56507 52.68240)
8068,8069,1,0.0,240.0,120.0,21,-1.550278,52.682343,POINT (-1.55028 52.68234)


In [80]:
# make a convex hull around the points, convert it to a geodataframe, and then save it
triangle=gdf.unary_union.convex_hull
hull = gpd.GeoDataFrame(geometry=gpd.GeoSeries(triangle))
hull
hull.set_crs('wgs84')
hull.to_file('goldentriangle.geojson',driver='GeoJSON')

In [ ]:
# testing out the centroid loop
population=0
for index,row in centroids.iterrows():
    if(row['geometry'].within(df['geometry'][0])):
        population=population+row['population']
# if(population/totalpop>0.9):
print(population/totalpop)

In [ ]:
# testing out the isochrone loop

for index,isochrone in df[0:2].iterrows():
    population=0
    for i,centroid in centroids.iterrows():
        if(centroid['geometry'].within(isochrone['geometry'])):
            population=population+centroid['population']
    print(index)
    if(population/totalpop>0.9):
        df['inside']=True
    else:
        df['inside']=False